## Add project root

In [ ]:
import sys
from pathlib import Path
import importlib

# Add the parent of the *outer* DLC-Jupyter-Notebooks folder to sys.path
project_root = Path().resolve().parents[0]  # This is /Users/atanugiri/Downloads/GhrelinBehaviorQuantification
print(project_root)
sys.path.append(str(project_root))


## Connect to db

In [ ]:
import psycopg2
import platform

host = "localhost" if platform.system() == "Windows" else "129.108.49.49"
conn = psycopg2.connect(
    dbname="deeplabcut_db", user="postgres", 
    password="1234", host=host, port="5432")
cursor = conn.cursor()


## Import modules

In [ ]:
import importlib
import Python_scripts.Data_analysis.fetch_id_list as fetch_mod
importlib.reload(fetch_mod)
from Python_scripts.Data_analysis.fetch_id_list import fetch_id_list


In [ ]:
import importlib
import Python_scripts.Feature_functions.stopping_points
import Python_scripts.Data_analysis.plot_groupwise_bar

importlib.reload(Python_scripts.Feature_functions.stopping_points)
importlib.reload(Python_scripts.Data_analysis.plot_groupwise_bar)

from Python_scripts.Feature_functions.stopping_points import (
    stopping_points_bbox_for_trial, batch_stopping_points_bbox
)
from Python_scripts.Data_analysis.plot_groupwise_bar import plot_groupwise_bar


## Fetch id list

In [ ]:
task_name = "AllTaskMinusFoodOnly"
bad_id = [549, 559, 566, 567, 570, 571, 595, 617, 621, 638, 640, 36]

saline_id, ghrelin_id, exc_id, inh_id = fetch_id_list(
    conn,
    task_name=['FoodLight', 'ToyOnly', 'ToyLight', 'LightOnly'],
    dose_mult=2,
    genotype="white",
    bad_ids=bad_id,
    csv_prefix="dlc_table",   # -> dlc_table_saline.csv, dlc_table_ghrelin.csv, ...
    min_trial_length=None     # or 600
)

print(f"saline_id: {saline_id}\n")
print(f"ghrelin_id: {ghrelin_id}\n")
print(f"exc_id: {exc_id}\n")
print(f"inh_id: {inh_id}\n")


In [ ]:
out = stopping_points_bbox_for_trial(452, conn)
for k, v in out.items():
    print(f"k = {k}, v = {v}")
# print(out)

## Calculate stopping points

### Batch call with different input parameter sweep

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# --- Configure once ---
bodypart = 'Midback'          # or 'Midback'
time_limit = None          # e.g., 120 for first 2 minutes; None for full trial
dx_thresh = dy_thresh = 0.1 # np.arange(0.05, 0.21, 0.05)
min_stop_duration_s = np.arange(0.5, 5.1, 0.5)

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    "Ghrelin": ghrelin_id,
    # "Inhibitory": inh_id,
    # "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

outfile = f"White_2X_{task_name}_{bodypart}_stop_pts_duration_sweep.pdf"
with PdfPages(outfile) as pdf:
    for duration in min_stop_duration_s:
        print(f"[INFO] Analyzing duration = {duration}")

        # Compute all groups for this window size
        frames = []
        for label, ids in group_specs.items():
            df = batch_stopping_points_bbox(
                conn, ids,
                bodypart=bodypart,
                time_limit=time_limit,
                dx_thresh=dx_thresh,
                dy_thresh=dy_thresh,
                min_stop_duration_s=duration,
            )
            # Keep what we need; add group label
            df = df[['trial_id', 'stops_per_min']].copy().dropna()
            df['group'] = label
            frames.append(df)

        if not frames:
            print("[WARN] No groups provided—skipping this window.")
            continue

        df_stop = pd.concat(frames, ignore_index=True)

        # Plot
        order = list(group_specs.keys())
        fig, ax = plot_groupwise_bar(
            df_stop,
            y='stops_per_min',
            ylabel='# of stops/min',
            plot_type='bar',
            show_points=False,
            order=order,
            show_stats=True,
            tests_to_show=("ranksums", "ttest", "anova")  # if supported
        )
        ax.set_title(f"{task_name} | {bodypart} | duration={duration}", pad=20)

        pdf.savefig(fig, bbox_inches='tight')
        plt.close(fig)

print(f"[✓] Saved {outfile}")


### Single call with fixed hyper parameters

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Configure once ---
bodypart = 'Midback'          # or 'Midback'
time_limit = None          # e.g., 120 for first 2 minutes; None for full trial
dx_thresh = dy_thresh = 0.1 # np.arange(0.05, 0.21, 0.05)
min_stop_duration_s = 1.5

# Put every group you *might* use here; leave others as None or [] when not needed.
group_specs = {
    "Saline":  saline_id,
    # "Ghrelin": ghrelin_id,
    "Inhibitory": inh_id,
    "Excitatory": exc_id,
}
# Keep only defined & non-empty groups
group_specs = {label: ids for label, ids in group_specs.items() if ids not in (None, [], ())}

frames = []
for label, ids in group_specs.items():
    df = batch_stopping_points_bbox(
        conn, ids,
        bodypart=bodypart,
        time_limit=time_limit,
        dx_thresh=dx_thresh,
        dy_thresh=dy_thresh,
        min_stop_duration_s=min_stop_duration_s,
    )
    # Keep what we need; add group label
    df = df[['trial_id', 'stops_per_min']].copy().dropna()
    df['group'] = label
    frames.append(df)

if not frames:
    print("[WARN] No groups provided—skipping this window.")

df_stop = pd.concat(frames, ignore_index=True)

# Plot
order = list(group_specs.keys())
fig, ax = plot_groupwise_bar(
    df_stop,
    y='stops_per_min',
    ylabel='# of stops/min',
    plot_type='bar',
    show_points=False,
    order=order,
    show_stats=True,
    tests_to_show=("ranksums", "ttest", "anova")  # if supported
)
ax.set_title(f"{task_name} | {bodypart} | duration={min_stop_duration_s}s", pad=20)

fig.savefig(f"White_Modulation_2X_{task_name}_stop_pt.pdf", dpi=300, bbox_inches='tight')
